#  Build an Interactive Dashboard with Ploty Dash

## Import Libraries 

In [1]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px


## Load and Prepare the Dataset

In [4]:

import requests
import pandas as pd
import io

# Define the URL
URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

# Fetch the dataset
response = requests.get(URL)
spacex_launch_dash_csv = io.BytesIO(response.content)
spacex_df = pd.read_csv(spacex_launch_dash_csv)
spacex_df.head()


,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


## Initialize Dash App


In [7]:
app = dash.Dash(__name__)
app.title = "SpaceX Launch Dashboard"


##  Build the App Layout

### TASK 1: Add a Launch Site Dropdown Input Component

In [8]:
app.layout = html.Div([
    html.H1("SpaceX Launch Records Dashboard", style={'textAlign': 'center'}),
    # Dropdown for Launch Sites
    dcc.Dropdown(
        id='site-dropdown',
        options=[
            {'label': 'All Sites', 'value': 'ALL'},
            *[{'label': site, 'value': site} for site in spacex_df['Launch Site'].unique()]
        ],
        value='ALL',
        placeholder="Select a Launch Site",
        searchable=True
    ),
    html.Br(),
    # Pie chart
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),
    # Payload range slider
    html.P("Payload range (Kg):"),
    dcc.RangeSlider(
        id='payload-slider',
        min=min_payload,
        max=max_payload,
        step=1000,
        marks={i: f'{i}' for i in range(int(min_payload), int(max_payload) + 1, 1000)},
        value=[min_payload, max_payload]
    ),
    # Scatter plot
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])


## Add Callback Functions

This callback updates the pie chart based on the dropdown value:

In [9]:
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value')
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        fig = px.pie(
            spacex_df, 
            names='Launch Site', 
            values='class', 
            title='Total Success Launches for All Sites'
        )
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        fig = px.pie(
            filtered_df, 
            names='class', 
            title=f"Total Success Launches for {selected_site}"
        )
    return fig


### TASK 4: Render Success-Payload-Scatter-Chart

In [10]:
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'), 
     Input('payload-slider', 'value')]
)
def update_scatter_chart(selected_site, payload_range):
    low, high = payload_range
    filtered_df = spacex_df[
        (spacex_df['Payload Mass (kg)'] >= low) & 
        (spacex_df['Payload Mass (kg)'] <= high)
    ]
    if selected_site == 'ALL':
        fig = px.scatter(
            filtered_df, 
            x='Payload Mass (kg)', 
            y='class', 
            color='Booster Version Category',
            title='Payload vs. Outcome for All Sites'
        )
    else:
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]
        fig = px.scatter(
            filtered_df, 
            x='Payload Mass (kg)', 
            y='class', 
            color='Booster Version Category',
            title=f"Payload vs. Outcome for {selected_site}"
        )
    return fig


## Run the Dash application:

In [11]:
if __name__ == '__main__':
    app.run_server(debug=True)
